In [1]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))

from filepath import *

import numpy as np
from math import floor
from random import sample

In [2]:
# def generate_prioritas(n_user,n_jurusan,n_prioritas):
#     dict_user = {}
#     tmp_2 = []
#     header = ['userId']

#     for i in n_user:
#         result = sample(range(0, n_jurusan), n_prioritas)
#         dict_user[i] = result
        
#     for key, value in dict_user.items():
#         count = len(value)
#         tmp = [key]
#         tmp.extend(value)
#         tmp_2.append(tmp)

#     for i in range(count):
#         prefix = "priority_"
#         name = prefix + str(i+1)
#         header.append(name)

#     df = pd.DataFrame(tmp_2,columns=header)
        
#     return df

def get_daya_tampung(data):

    #define priority
    header = data.columns.tolist()

    unwanted = {'userId', 'rerata'}
    header = [e for e in header if e not in unwanted]

    result = []

    #get data jurusan
    kuota = load_file('kuota kampus','rasionalisasi\\kuota kampus')
    #     kuota['Peminat'] = kuota['Peminat'].apply(replace)
    kuota['batas'] = 500 #Nilai minimal

    for m,n in enumerate(header):
        #get pendaftar
        data_prior = data[n].value_counts()
        data_prior = data_prior.sort_index(axis=0)
        list_jurusan = data_prior.index.tolist()
        jumlah_pendaftar = data_prior.values.tolist()

        for i,j in zip(list_jurusan,jumlah_pendaftar):
            for k,l in kuota.iterrows():                
                if k == i:
                    daya_tampung = l['Daya Tampung']
                    #if peminat 0
                    if j >= l.Peminat | l.Peminat == 0:
                        daya_tampung_new = daya_tampung

                    #normal
                    else:          
                        daya_tampung_new = floor((j/l.Peminat)*daya_tampung)

                    #if treshold 0
                    if daya_tampung_new == 0:
                        daya_tampung_new = daya_tampung

            tmp = list([i,daya_tampung_new,l.batas,m+1])

            result.append(tmp)

        df = pd.DataFrame(result, columns =['jurusanId','dayatampungBaru','nilaiMinimal','priority'])
        
    return df

def peringkat(rerata,teratas_10_persen,rerata_total):
    if rerata > teratas_10_persen:
        return "Peringkat 10 persen pendaftar"
    elif rerata > rerata_total:
        return "Nilai di atas rata-rata pendaftar"
    else:
        return "Nilai di bawah rata-rata pendaftar"
    
def ranking_pilihan(data_total,data_kuota):
    result = {}
    
    #Get Jurusan ID
    jurusan = data_kuota.jurusanId.to_list()
    
    #Each Jurusan ID
    for i in jurusan:
        list_user = []
        #Get Treshold
        minim = data_kuota.loc[data_kuota.jurusanId==i].nilaiMinimal.values[0]
        kuota = data_kuota.loc[data_kuota.jurusanId==i].dayatampungBaru.values[0]
        
        #Get User by Pilihan
        user = data_total.loc[data_total.pilihan_1==i]
        
        #Sort by rerata
        user = user.sort_values('rerata', ascending = False)
        
        #if surpass treshold
        for index,j in user.iterrows():
            if j.rerata > minim:
                list_user.append(j.userId)
                result[i] = list_user[:kuota] #Limit by kuota
            
                
    return result

In [3]:
##Get data 

##Count Statistik
result = get_daya_tampung(data) #get data baru jurusan

# #Get peringkat
data['peringkat'] = data.apply(lambda x: peringkat(x.rerata,top_10,rerata_total),axis=1)

#User lolos per jurusan
tmp = ranking_pilihan(data,result)

NameError: name 'generate_nilai' is not defined

In [ ]:
#Search User
userId = 1
[k for k, v in tmp.items() if userId in v]